In [52]:
import pandas as pd

df = pd.read_csv('../data/samples/sample.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138614 entries, 0 to 138613
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           138614 non-null  object 
 1   timestamp                    138614 non-null  float64
 2   hour                         138614 non-null  int64  
 3   day                          138614 non-null  int64  
 4   month                        138614 non-null  int64  
 5   datetime                     138614 non-null  object 
 6   timezone                     138614 non-null  object 
 7   source                       138614 non-null  object 
 8   destination                  138614 non-null  object 
 9   cab_type                     138614 non-null  object 
 10  product_id                   138614 non-null  object 
 11  name                         138614 non-null  object 
 12  price                        127622 non-null  float64
 13 

In [53]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138614 entries, 0 to 138613
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           138614 non-null  object 
 1   timestamp                    138614 non-null  float64
 2   hour                         138614 non-null  int64  
 3   day                          138614 non-null  int64  
 4   month                        138614 non-null  int64  
 5   datetime                     138614 non-null  object 
 6   timezone                     138614 non-null  object 
 7   source                       138614 non-null  object 
 8   destination                  138614 non-null  object 
 9   cab_type                     138614 non-null  object 
 10  product_id                   138614 non-null  object 
 11  name                         138614 non-null  object 
 12  price                        127622 non-null  float64
 13 

In [54]:
missing_values = df.isnull().sum()
print(missing_values)

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          10992
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

In [55]:
df = df.dropna(subset=['price'])

In [56]:
missing_values = df.isnull().sum()
print(missing_values)

id                             0
timestamp                      0
hour                           0
day                            0
month                          0
datetime                       0
timezone                       0
source                         0
destination                    0
cab_type                       0
product_id                     0
name                           0
price                          0
distance                       0
surge_multiplier               0
latitude                       0
longitude                      0
temperature                    0
apparentTemperature            0
short_summary                  0
long_summary                   0
precipIntensity                0
precipProbability              0
humidity                       0
windSpeed                      0
windGust                       0
windGustTime                   0
visibility                     0
temperatureHigh                0
temperatureHighTime            0
temperatur

In [57]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['day_of_week'] = df['datetime'].dt.dayofweek
print(df['day_of_week'].head())


0    6
1    1
2    2
3    4
4    3
Name: day_of_week, dtype: int32


In [58]:
preliminary_features_name = ['hour', 'month', 'source', 'destination',
       'name', 'distance', 'surge_multiplier', 'apparentTemperature', 'short_summary', 'windSpeed', 'visibility',
       'day_of_week']

In [59]:
features = df[preliminary_features_name]
target = df['price']

In [60]:
from sklearn.preprocessing import OneHotEncoder

def encode_features_one_hot(dataframe, features_names, encoder):
    new_features = encoder.transform(dataframe[features_names])
    new_columns_df = pd.DataFrame(new_features, columns = encoder.get_feature_names_out(features_names))
    new_dataframe = pd.concat([dataframe.reset_index(), new_columns_df.reset_index()], axis=1)
    new_dataframe.drop(features_names, axis=1, inplace=True)
    return new_dataframe

# Encoding features
encoder = OneHotEncoder(sparse_output=False)

# Select columns that should be encoded (values are not numbers)
features_names = list(features.select_dtypes(exclude='number').columns)

encoder.fit(features[features_names])
features = encode_features_one_hot(features, features_names, encoder)

In [61]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(features)


features = pd.DataFrame(scaler.transform(features), columns = features.columns)

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [63]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

LinearRegression()

In [64]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

y_pred = linear_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared Score: {r2}')

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE: {mape}')


Mean Squared Error: 6.299431179692439
R-squared Score: 0.9277079060352151
MAPE: 0.13470746427456395
